In [9]:
import pyarrow.parquet as pq

table = pq.read_table(parquet_file)

df = table.to_pandas()

json_data = df.to_json(orient='records')

with open('../../data/raw/train-00000-of-00001-dcec52ec7fe8275d.json', 'w') as f:
    f.write(json_data)


In [4]:
import pandas as pd
data = pd.read_json("../../data/raw/cocktail_recipes.json")
data

,title,ingredients,directions,misc,source,ner
0,Mimosa,"[90 ml Champagne, 90 ml Orange juice]",[1 Champagne: Slowly pour the Champagne into y...,[With a 50/50 mix of Champagne and Orange Juic...,cocktailclub,"[champagne, orange juice]"
1,Green Coconut,"[1 Bars spoon Matcha tea powder, 4 Bar spoons ...","[1 Prepare: With your Fruit Zester, peel off a...",[This creamy and sweet detox mocktail is a hea...,cocktailclub,[]
